## 主要实现
- 批量查询位置点 经纬度等信息
- 经纬度转换为 geohash块、
- 目标geohash 临近geohash块
    - 九宫格

### Note
- 查询位置列表， 需<font color=#FF1953>提前提供</font>

In [5]:
city_lis=['北京', '上海', '郑州']

### 获取位置地点经纬度等信息
- 获取经纬度等信息
- 计算 geohash

In [6]:
import pandas as pd
import mzgeohash
import urllib2
import urllib
import json

In [23]:
'''
根据位置名称，获取经纬度等信息
'''
def get_metas_from_baidu(city):
    metas={}
    metas['key']='f247cdb592eb43ebac6ccd27f796e2d2'
    metas['address']=city   # 城市名称
    
    data=urllib.urlencode(metas)
    url='http://api.map.baidu.com/geocoder?output=json&'+urllib.urlencode(metas)
    # 注释 url='http://api.map.baidu.com/geocoder?'+urllib.urlencode(metas)+'&output=json'
    unicode_s=urllib2.urlopen(url)
    
    return json.loads(unicode_s.read())

city_meta_lis=[]  #  计算 geohash 编码并存储
for city in city_lis:
    tmp=get_metas_from_baidu(city)
    latitude=tmp['result']['location']['lat']
    longitude=tmp['result']['location']['lng']
    t_geohash=mzgeohash.encode((longitude,latitude)) # 使用mzgeohash 计算geohash
    city_meta_lis.append([city,latitude,longitude,t_geohash]) # 存储

In [24]:
# 格式化输出
df=pd.DataFrame(city_meta_lis, columns=['city','latitude','longitude','geohash'])
df

,city,latitude,longitude,geohash
0,北京,39.929986,116.395645,wx4g0th9p0gk
1,上海,31.249162,121.487899,wtw3u88z94p0
2,郑州,34.756610,113.649644,ww0vdpjputsw


### mzgeohash
- 调用方式
- 实现原理

### list all member methods

In [23]:
dir(mzgeohash)

['__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 '__path__',
 '__version__',
 'adjacent',
 'decode',
 'encode',
 'geohash',
 'neighbors',
 'neighborsfit']

#### 经纬度 转换为 geohash

In [31]:
mzgeohash.encode([139.76608408614993, 35.681382017210126])

'xn76urwe1g9y'

#### geohash 转换为 经纬度

In [33]:
mzgeohash.decode('xn76urwe1g9y')

(139.76608408614993, 35.681382017210126)

#### 已知 geohash ，计算某一方向的临近geohash
- 参数 1
    - 已知 geohash 串
- 参数 2
    - 方向

In [35]:
mzgeohash.adjacent('xn76urwe1g9y','n') # 

'xn76urwe1g9z'

#### 已知 geohash ，临近geohash
- 九宫格，即周围 8 个geohash

In [34]:
mzgeohash.neighbors('xn76urwe1g9y')

{'c': 'xn76urwe1g9y',
 'e': 'xn76urwe1gdn',
 'n': 'xn76urwe1g9z',
 'ne': 'xn76urwe1gdp',
 'nw': 'xn76urwe1g9x',
 's': 'xn76urwe1g9v',
 'se': 'xn76urwe1gdj',
 'sw': 'xn76urwe1g9t',
 'w': 'xn76urwe1g9w'}

#### 已知 某位置点经纬度和周围经纬度点集，计算同属 geohash

In [37]:
test_centroid = [-122.18472385000001, 37.7881345]
test_points = [
(-122.2992715, 37.9030588),
(-122.396742, 37.792976),
(-122.4474142, 37.72198087),
(-121.9764, 37.557355),
(-122.029095, 37.973737),
(-122.224274, 37.774963),
(-122.271604, 37.803664),
(-122.126871, 37.697185),
(-122.087967, 37.670399),
(-122.123801, 37.893394),
(-122.269029, 37.80787),
(-122.265609, 37.797484),
(-122.267227, 37.828415),
(-122.067423, 37.905628),
(-122.267227, 37.828415),
(-122.38666, 37.599787),
(-122.075567, 37.690754),
(-122.401407, 37.789256),
(-122.283451, 37.87404),
(-122.269029, 37.80787),
(-122.1837911, 37.87836087),
(-122.419694, 37.765062),
(-122.2945822, 37.80467476),
(-122.21244024, 37.71297174),
(-121.945154, 38.018914),
(-122.466233, 37.684638),
(-122.056013, 37.928403),
(-122.406857, 37.784991),
(-122.418466, 37.752254),
(-122.26978, 37.853024),
(-122.251793, 37.844601),
(-121.928099, 37.699759),
(-122.416038, 37.637753),
(-122.1613112, 37.72261921),
(-122.0575506, 37.63479954),
(-122.392612, 37.616035),
(-122.413756, 37.779528),
(-122.353165, 37.936887),
(-122.197273, 37.754006),
(-122.017867, 37.591208),
(-122.024597, 38.003275),
(-122.4690807, 37.70612055),
(-122.268045, 37.869867),
(-122.444116, 37.664174),
(-121.900367, 37.701695),
(-122.317269, 37.925655),
(-122.434092, 37.732921)
]

# expect = '9q9'
mzgeohash.neighborsfit(test_centroid, test_points)

'9q9'

### 使用unittest进行 mzgeohash 模块测试
- 通过 testsuit 执行测试用例
- TestCase：所有测试用例的基本类，给定测试方法的名称，返回测试用例实例；
- TestSuit：组织测试用例的实例，支持测试用例的添加和删除，最终将传递给 testRunner进行测试执行；
- TextTestRunner：进行测试用例执行的实例，其中Text的意思是以文本形式显示测试结果。
    - 测试结果保存在 TextTestResult 实例中，包括运行多少测试用例，成功多少，失败多少等信息；

In [6]:
import unittest
from mzgeohash.test_geohash import Test_encode_decode
from mzgeohash.test_geohash import Test_adjacent
from mzgeohash.test_geohash import Test_neighbors
from mzgeohash.test_geohash import Test_neighborsfit

In [35]:
suite=unittest.TestSuite()
suite.addTest(Test_encode_decode('test_decode'))
suite.addTest(Test_encode_decode('test_encode'))
suite.addTest(Test_encode_decode('test_roundtrip'))
suite.addTest(Test_neighbors('test_neighbors'))
suite.addTest(Test_adjacent('test_adjacent'))
suite.addTest(Test_neighborsfit('test_neighborsfit'))

runner=unittest.TextTestRunner()
runner.run(suite);

............
----------------------------------------------------------------------
Ran 12 tests in 0.053s

OK
